In [1]:
# import logging
# import sys
# from pathlib import Path
# from FileUtils import FileUtils


In [2]:
# # Add project root to Python path if needed
# project_root = str(Path().resolve().parent)
# if project_root not in sys.path:
#     sys.path.append(project_root)

In [4]:
# AZURE_STORAGE_CONNECTION_STRING="DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName=rifeedbackml7534979049;AccountKey=F3QkZr5mbjiitdgfU4npaINOYLl2QE3Uo/9A2BfKp4KEaNXjfxPYJzyyeN0t8tigV6mL/s3idi3/+AStRy93Lw==;BlobEndpoint=https://rifeedbackml7534979049.blob.core.windows.net/;FileEndpoint=https://rifeedbackml7534979049.file.core.windows.net/;QueueEndpoint=https://rifeedbackml7534979049.queue.core.windows.net/;TableEndpoint=https://rifeedbackml7534979049.table.core.windows.net/"

# file_utils_local = FileUtils(storage_type="local", connection_string=AZURE_STORAGE_CONNECTION_STRING)

2024-12-08 16:53:33,581 - FileUtils.core.file_utils - INFO - Project root: /mnt/batch/tasks/shared/LS_root/mounts/clusters/basic-cpu/code/Users/topi.jarvinen/semantic-text-analyzer
2024-12-08 16:53:33,698 - FileUtils.core.file_utils - INFO - FileUtils initialized with local storage


In [5]:
# import logging
# import sys
# from pathlib import Path

# # Remove any existing handlers
# logger = logging.getLogger()
# for handler in logger.handlers:
#     logger.removeHandler(handler)

# # Create and configure a new handler
# handler = logging.StreamHandler(sys.stdout)
# handler.setLevel(logging.DEBUG)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)

# # Configure root logger
# logger.setLevel(logging.DEBUG)
# logger.addHandler(handler)

# # Configure specific logger for setup helper
# setup_logger = logging.getLogger("scripts.azure.setup_helpers")
# setup_logger.setLevel(logging.DEBUG)
# setup_logger.addHandler(handler)

# # Get project root (you already have this)
# project_root = Path().resolve().parent

# from scripts.azure.setup_helpers import setup_voikko

# # Run setup with explicit project root
# if setup_voikko(project_root=project_root):
#     print("Voikko ready")
# else:
#     print("Using fallback mode")

2024-12-08 16:53:39,337 - scripts.azure.setup_helpers - ERROR - Setup script not found at /mnt/batch/tasks/shared/LS_root/mounts/clusters/basic-cpu/code/Users/topi.jarvinen/semantic-text-analyzer/scripts/azure/setup_voikko.sh
2024-12-08 16:53:39,337 - scripts.azure.setup_helpers - ERROR - Setup script not found at /mnt/batch/tasks/shared/LS_root/mounts/clusters/basic-cpu/code/Users/topi.jarvinen/semantic-text-analyzer/scripts/azure/setup_voikko.sh
Using fallback mode


In [7]:
import libvoikko

In [47]:
dir(libvoikko)

['CDLL',
 'Dictionary',
 'GrammarError',
 'MAX_ANALYSIS_COUNT',
 'MAX_WORD_CHARS',
 'POINTER',
 'Sentence',
 'Structure',
 'SuggestionStrategy',
 'Token',
 'Voikko',
 'VoikkoException',
 'VoikkoLibrary',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_anyStringToPath',
 '_anyStringToUtf8',
 '_boolToInt',
 'addressof',
 'binary_str',
 'byref',
 'c_char',
 'c_char_p',
 'c_int',
 'c_size_t',
 'c_void_p',
 'c_wchar',
 'c_wchar_p',
 'create_unicode_buffer',
 'os',
 'platform',
 'pointer',
 'repr_conv',
 'sizeof',
 'string_at',
 'sys',
 'unicode_literals',
 'unicode_str']

In [46]:
from libvoikko import Voikko

In [48]:
import libvoikko

In [49]:
libvoikko.Voikko("fi")

In [50]:
# Try to initialize Voikko

v = libvoikko.Voikko("fi")
# Test with a simple word
result = v.analyze("kissa")
print(f"Voikko test result: {result}")

    # v.terminate()
    

Voikko test result: [{'BASEFORM': 'kissa', 'CLASS': 'nimisana', 'FSTOUTPUT': '[Ln][Xp]kissa[X]kiss[Sn][Ny]a', 'NUMBER': 'singular', 'SIJAMUOTO': 'nimento', 'STRUCTURE': '=ppppp', 'WORDBASES': '+kissa(kissa)'}]


In [2]:
import os
import logging
from pathlib import Path
import libvoikko

In [3]:
# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [40]:
def configure_voikko():
    """Configure Voikko paths in Azure environment."""
    try:
        conda_prefix = os.environ.get('CONDA_PREFIX')
        if not conda_prefix:
            print("No CONDA_PREFIX found")
            return False

        # Set up library path
        lib_path = Path(conda_prefix) / 'lib'
        if lib_path.exists():
            if 'LD_LIBRARY_PATH' in os.environ:
                os.environ['LD_LIBRARY_PATH'] = f"{lib_path}:{os.environ['LD_LIBRARY_PATH']}"
            else:
                os.environ['LD_LIBRARY_PATH'] = str(lib_path)
            print(f"Set LD_LIBRARY_PATH to include {lib_path}")

        # Set up dictionary path
        dict_path = Path(conda_prefix) / 'share' / 'voikko'
        if dict_path.exists():
            os.environ['VOIKKO_DICTIONARY_PATH'] = str(dict_path)
            print(f"Set VOIKKO_DICTIONARY_PATH to {dict_path}")
        else:
            print(f"Dictionary path not found at {dict_path}")

        # Check for library file
        lib_file = lib_path / 'libvoikko.so.1'
        if not lib_file.exists():
            lib_file_alt = lib_path / 'libvoikko.so'
            if lib_file_alt.exists():
                # Create symbolic link
                try:
                    lib_file.symlink_to(lib_file_alt)
                    print(f"Created symbolic link from {lib_file_alt} to {lib_file}")
                except Exception as e:
                    print(f"Could not create symbolic link: {e}")

        # Try to initialize Voikko
        try:
            v = libvoikko.Voikko("fi")
            # Test with a simple word
            result = v.analyze("kissa")
            print(f"Voikko test result: {result}")
            v.terminate()
            return True
        except Exception as e:
            print(f"Failed to initialize Voikko: {e}")
            return False

    except Exception as e:
        print(f"Error configuring Voikko: {e}")
        return False

In [41]:
import os
import logging
from pathlib import Path
import libvoikko

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def configure_voikko():
    """Configure Voikko paths in Azure environment."""
    try:
        conda_prefix = os.environ.get('CONDA_PREFIX')
        if not conda_prefix:
            print("No CONDA_PREFIX found")
            return False

        # Set up library path
        lib_path = Path(conda_prefix) / 'lib'
        if lib_path.exists():
            if 'LD_LIBRARY_PATH' in os.environ:
                os.environ['LD_LIBRARY_PATH'] = f"{lib_path}:{os.environ['LD_LIBRARY_PATH']}"
            else:
                os.environ['LD_LIBRARY_PATH'] = str(lib_path)
            print(f"Set LD_LIBRARY_PATH to include {lib_path}")

        # Set up dictionary path
        dict_path = Path(conda_prefix) / 'share' / 'voikko'
        if dict_path.exists():
            os.environ['VOIKKO_DICTIONARY_PATH'] = str(dict_path)
            print(f"Set VOIKKO_DICTIONARY_PATH to {dict_path}")
        else:
            print(f"Dictionary path not found at {dict_path}")

        # Check for library file
        lib_file = lib_path / 'libvoikko.so.1'
        if not lib_file.exists():
            lib_file_alt = lib_path / 'libvoikko.so'
            if lib_file_alt.exists():
                # Create symbolic link
                try:
                    lib_file.symlink_to(lib_file_alt)
                    print(f"Created symbolic link from {lib_file_alt} to {lib_file}")
                except Exception as e:
                    print(f"Could not create symbolic link: {e}")

        # Try to initialize Voikko
        try:
            v = libvoikko.Voikko("fi")
            # Test with a simple word
            result = v.analyze("kissa")
            print(f"Voikko test result: {result}")
            v.terminate()
            return True
        except Exception as e:
            print(f"Failed to initialize Voikko: {e}")
            return False

    except Exception as e:
        print(f"Error configuring Voikko: {e}")
        return False

# Try to configure Voikko
if configure_voikko():
    print("Voikko configured successfully")
else:
    print("Using fallback mode")

No CONDA_PREFIX found
Using fallback mode


In [7]:
# Try to configure Voikko
if configure_voikko():
    print("Voikko configured successfully")
else:
    print("Using fallback mode")

No CONDA_PREFIX found
Using fallback mode


In [8]:
import os
import sys
import logging
from pathlib import Path

# Setup logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Print environment information
print("Environment Information:")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print("\nRelevant Environment Variables:")
for var in ['CONDA_PREFIX', 'CONDA_DEFAULT_ENV', 'CONDA_EXE', 'PATH', 'LD_LIBRARY_PATH']:
    print(f"{var}: {os.environ.get(var, 'Not set')}")

# Try to find conda-related paths
python_path = Path(sys.executable)
possible_conda_prefix = python_path.parent.parent
print(f"\nPossible conda prefix from Python path: {possible_conda_prefix}")

# Check if this looks like a conda environment
has_conda_meta = (possible_conda_prefix / "conda-meta").exists()
has_lib = (possible_conda_prefix / "lib").exists()
print(f"Has conda-meta directory: {has_conda_meta}")
print(f"Has lib directory: {has_lib}")

# Check for Voikko files
lib_dir = possible_conda_prefix / "lib"
voikko_files = list(lib_dir.glob("*voikko*")) if lib_dir.exists() else []
print("\nFound Voikko files:")
for f in voikko_files:
    print(f"  {f}")

Environment Information:
Python executable: /anaconda/envs/semantic-analyzer/bin/python
Python version: 3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:51:40) 
[GCC 13.3.0]

Relevant Environment Variables:
CONDA_PREFIX: Not set
CONDA_DEFAULT_ENV: Not set
CONDA_EXE: Not set
PATH: /anaconda/envs/azureml_py38/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
LD_LIBRARY_PATH: Not set

Possible conda prefix from Python path: /anaconda/envs/semantic-analyzer
Has conda-meta directory: True
Has lib directory: True

Found Voikko files:


In [12]:
import os
import logging
from pathlib import Path
import libvoikko

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def configure_voikko_azure():
    """Configure Voikko in Azure ML environment."""
    try:
        # Use the actual conda environment path we found
        conda_env_path = Path("/anaconda/envs/semantic-analyzer")
        print(f"Using conda environment path: {conda_env_path}")

        # Set up library path
        lib_path = conda_env_path / 'lib'
        if lib_path.exists():
            if 'LD_LIBRARY_PATH' in os.environ:
                os.environ['LD_LIBRARY_PATH'] = f"{lib_path}:{os.environ['LD_LIBRARY_PATH']}"
            else:
                os.environ['LD_LIBRARY_PATH'] = str(lib_path)
            print(f"Set LD_LIBRARY_PATH to include {lib_path}")

        # Set up dictionary path and check for dictionaries
        dict_paths = [
            conda_env_path / 'share' / 'voikko',
            conda_env_path / 'lib' / 'voikko',
            Path('/usr/share/voikko'),
            Path('/usr/lib/voikko')
        ]

        for dict_path in dict_paths:
            if dict_path.exists():
                os.environ['VOIKKO_DICTIONARY_PATH'] = str(dict_path)
                print(f"Found and set VOIKKO_DICTIONARY_PATH to {dict_path}")
                break
        else:
            logger.warning("No dictionary path found")

        # Try to initialize Voikko with explicit path
        try:
            v = libvoikko.Voikko("fi", str(lib_path))
            # Test with a simple word
            result = v.analyze("kissa")
            print(f"Voikko test result: {result}")
            v.terminate()
            logger.info("Voikko initialized successfully")
            return True
        except Exception as e:
            print(f"Failed to initialize Voikko: {e}")
            # Try without explicit path
            try:
                v = libvoikko.Voikko("fi")
                result = v.analyze("kissa")
                print(f"Voikko test result (without explicit path): {result}")
                v.terminate()
                logger.info("Voikko initialized successfully without explicit path")
                return True
            except Exception as e2:
                print(f"Failed to initialize Voikko without explicit path: {e2}")
                return False

    except Exception as e:
        print(f"Error configuring Voikko: {e}")
        return False

# Try to configure Voikko
if configure_voikko_azure():
    print("Voikko configured successfully")
else:
    print("Using fallback mode")

Using conda environment path: /anaconda/envs/semantic-analyzer
Set LD_LIBRARY_PATH to include /anaconda/envs/semantic-analyzer/lib
Failed to initialize Voikko: libvoikko.so.1: cannot open shared object file: No such file or directory
Failed to initialize Voikko without explicit path: libvoikko.so.1: cannot open shared object file: No such file or directory
Using fallback mode


In [14]:
from libvoikko import Voikko

In [19]:
Voikko.voikkoInit()

AttributeError: type object 'Voikko' has no attribute 'voikkoInit'

In [20]:
import libvoikko

In [26]:
dir(libvoikko)

['CDLL',
 'Dictionary',
 'GrammarError',
 'MAX_ANALYSIS_COUNT',
 'MAX_WORD_CHARS',
 'POINTER',
 'Sentence',
 'Structure',
 'SuggestionStrategy',
 'Token',
 'Voikko',
 'VoikkoException',
 'VoikkoLibrary',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_anyStringToPath',
 '_anyStringToUtf8',
 '_boolToInt',
 'addressof',
 'binary_str',
 'byref',
 'c_char',
 'c_char_p',
 'c_int',
 'c_size_t',
 'c_void_p',
 'c_wchar',
 'c_wchar_p',
 'create_unicode_buffer',
 'os',
 'platform',
 'pointer',
 'repr_conv',
 'sizeof',
 'string_at',
 'sys',
 'unicode_literals',
 'unicode_str']

In [21]:
dir(libvoikko)

['CDLL',
 'Dictionary',
 'GrammarError',
 'MAX_ANALYSIS_COUNT',
 'MAX_WORD_CHARS',
 'POINTER',
 'Sentence',
 'Structure',
 'SuggestionStrategy',
 'Token',
 'Voikko',
 'VoikkoException',
 'VoikkoLibrary',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_anyStringToPath',
 '_anyStringToUtf8',
 '_boolToInt',
 'addressof',
 'binary_str',
 'byref',
 'c_char',
 'c_char_p',
 'c_int',
 'c_size_t',
 'c_void_p',
 'c_wchar',
 'c_wchar_p',
 'create_unicode_buffer',
 'os',
 'platform',
 'pointer',
 'repr_conv',
 'sizeof',
 'string_at',
 'sys',
 'unicode_literals',
 'unicode_str']

In [16]:
dir(Voikko)

['_Voikko__getGrammarError',
 '_Voikko__getLib',
 '_Voikko__grammarParagraph',
 '_Voikko__isValidInput',
 '_Voikko__listSupportedLanguagesForOperation',
 '_Voikko__splitTokens',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_sharedLibrarySearchPath',
 'analyze',
 'attributeValues',
 'getHyphenationPattern',
 'getVersion',
 'grammarErrors',
 'hyphenate',
 'listDicts',
 'listSupportedGrammarCheckingLanguages',
 'listSupportedHyphenationLanguages',
 'listSupportedSpellingLanguages',
 'sentences',
 'setAcceptAllUppercase',
 'setAcceptBulletedListsInGc',
 'setAcceptExtraHyphens',
 'setAcceptFirstUppercase',
 'setAcceptMissingHyphens',
 'setAcceptTitlesInGc',


In [30]:
import logging
from libvoikko import Voikko
from pathlib import Path

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def try_voikko_initialization():
    """Try different approaches to initialize Voikko."""
    
    conda_env_path = Path("/anaconda/envs/semantic-analyzer")
    possible_dict_paths = [
        conda_env_path / 'share' / 'voikko',
        conda_env_path / 'lib' / 'voikko',
        Path('/usr/share/voikko'),
        Path('/usr/lib/voikko')
    ]

    print("Checking dictionary paths:")
    for path in possible_dict_paths:
        print(f"Checking {path}")
        if path.exists():
            print(f"Found directory: {path}")
            # List contents
            print("Contents:")
            for item in path.iterdir():
                print(f"  {item}")

    try:
        # Try setting library search path first
        Voikko.setLibrarySearchPath(str(conda_env_path / 'lib'))
        
        # Try different initialization approaches
        approaches = [
            ("Default initialization", lambda: Voikko("fi")),
            ("With conda lib path", lambda: Voikko("fi", str(conda_env_path / 'lib'))),
            ("With share/voikko path", lambda: Voikko("fi", str(conda_env_path / 'share/voikko'))),
            ("With usr/share/voikko path", lambda: Voikko("fi", "/usr/share/voikko")),
            ("With usr/lib/voikko path", lambda: Voikko("fi", "/usr/lib/voikko")),
        ]

        for desc, init_func in approaches:
            try:
                print(f"\nTrying {desc}")
                v = init_func()
                result = v.analyze("kissa")
                print(f"Success! Analysis result: {result}")
                v.terminate()
                return True, desc
            except Exception as e:
                print(f"Failed: {str(e)}")
                continue

        print("All initialization attempts failed")
        return False, "All attempts failed"

    except Exception as e:
        print(f"Unexpected error: {e}")
        return False, str(e)

# Try initialization
success, method = try_voikko_initialization()
if success:
    print(f"Successfully initialized Voikko using: {method}")
else:
    print(f"Failed to initialize Voikko: {method}")

Checking dictionary paths:
Checking /anaconda/envs/semantic-analyzer/share/voikko
Checking /anaconda/envs/semantic-analyzer/lib/voikko
Checking /usr/share/voikko
Checking /usr/lib/voikko

Trying Default initialization
Failed: libvoikko.so.1: cannot open shared object file: No such file or directory

Trying With conda lib path
Failed: libvoikko.so.1: cannot open shared object file: No such file or directory

Trying With share/voikko path
Failed: libvoikko.so.1: cannot open shared object file: No such file or directory

Trying With usr/share/voikko path
Failed: libvoikko.so.1: cannot open shared object file: No such file or directory

Trying With usr/lib/voikko path
Failed: libvoikko.so.1: cannot open shared object file: No such file or directory
All initialization attempts failed
Failed to initialize Voikko: All attempts failed


In [37]:
Path(os.environ.get('HOME', '')) /'cloudfiles' / 'code' / 'Users' / 'topi.jarvinen'

PosixPath('/home/azureuser/cloudfiles/code/Users/topi.jarvinen')

In [ ]:
/home/azureuser/cloudfiles/code/Users/topi.jarvinen/

In [39]:
import os
import logging
from pathlib import Path
from libvoikko import Voikko

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

def setup_voikko():
    """Set up Voikko with dictionary paths based on official search order."""
    conda_env_path = Path("/anaconda/envs/semantic-analyzer")
    home_dir = Path(os.environ.get('HOME', '')) /'cloudfiles' / 'code' / 'Users' / 'topi.jarvinen'
    # home_dir = Path(os.environ.get('HOME', '')) /
    print(home_dir)
    
    # Dictionary search paths in order of precedence
    dict_paths = [
        # 1. User's .voikko directory
        home_dir / '.voikko',
        # 2. System-wide configuration
        Path('/etc/voikko'),
        # 3. Conda environment paths
        conda_env_path / 'share' / 'voikko',
        conda_env_path / 'lib' / 'voikko',
    ]

    # Debug: Print all potential paths
    print("Checking dictionary paths:")
    for path in dict_paths:
        print(f"Checking {path}")
        if path.exists():
            print(f"Found directory: {path}")
            # List contents with version subdirectory
            version_path = path / "5"  # Voikko appends "/2" for the dictionary version
            if version_path.exists():
                print(f"Found version directory: {version_path}")
                print("Contents:")
                for item in version_path.iterdir():
                    print(f"  {item}")

    # Create dictionary path string with OS-specific separator
    dict_path_str = os.pathsep.join(str(p) for p in dict_paths if p.exists())
    if dict_path_str:
        print(f"Setting VOIKKO_DICTIONARY_PATH to: {dict_path_str}")
        os.environ['VOIKKO_DICTIONARY_PATH'] = dict_path_str

    try:
        # Check available dictionaries
        print("Available dictionaries:")
        dicts = Voikko.listDicts()
        for d in dicts:
            print(f"  {d.variant} ({d.language})")

        # Try to initialize Voikko
        print("Attempting to initialize Voikko...")
        v = Voikko("fi")
        
        # Test with a simple word
        result = v.analyze("kissa")
        print(f"Analysis result: {result}")
        
        # Cleanup
        v.terminate()
        return True

    except Exception as e:
        print(f"Failed to initialize Voikko: {e}")
        return False

# Try setup
if setup_voikko():
    print("Voikko configured successfully")
else:
    print("Using fallback mode")

# Additional check: List supported languages
print("\nSupported spelling languages:", Voikko.listSupportedSpellingLanguages())
print("Supported dictionaries:", Voikko.listDicts())

/home/azureuser/cloudfiles/code/Users/topi.jarvinen
Checking dictionary paths:
Checking /home/azureuser/cloudfiles/code/Users/topi.jarvinen/.voikko
Found directory: /home/azureuser/cloudfiles/code/Users/topi.jarvinen/.voikko
Found version directory: /home/azureuser/cloudfiles/code/Users/topi.jarvinen/.voikko/5
Contents:
  /home/azureuser/cloudfiles/code/Users/topi.jarvinen/.voikko/5/mor-standard
Checking /etc/voikko
Checking /anaconda/envs/semantic-analyzer/share/voikko
Checking /anaconda/envs/semantic-analyzer/lib/voikko
Setting VOIKKO_DICTIONARY_PATH to: /home/azureuser/cloudfiles/code/Users/topi.jarvinen/.voikko
Available dictionaries:
Failed to initialize Voikko: libvoikko.so.1: cannot open shared object file: No such file or directory
Using fallback mode


OSError: libvoikko.so.1: cannot open shared object file: No such file or directory